In [1]:
from sqlalchemy import create_engine, Column, Date, String, Integer, Float, exc, and_, or_
from sqlalchemy.orm import Session, declarative_base

import requests

from selectorlib import Extractor
import requests
import json
from time import sleep
import random

In [2]:
def scrape(url, file):
    e = Extractor.from_yaml_file(file)
    headers = {
        'authority': 'www.amazon.com',
        'pragma': 'no-cache',
        'cache-control': 'no-cache',
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'none',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-dest': 'document',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }
    # Download the page using requests
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    if r.status_code > 500:
        return None

    # Pass the HTML of the page and create
    return e.extract(r.text)

In [3]:
engine = create_engine("sqlite+pysqlite:///books.sqlite")
Base = declarative_base()
Base.metadata.create_all(engine)
session = Session(engine)

In [4]:

class Amazon(Base):
    __tablename__ = "Amazon"

    isbn = Column(String, primary_key=True, nullable=False)
    rating = Column(Float, nullable=True)
    review_count = Column(Integer, nullable=True)
    url = Column(String, nullable=True)

    def __repr__(self) -> str:
        return (
            f"Amazon(isbn={self.isbn!r}, "
            f"rating={self.rating!r}, "
            f"review_count={self.review_count!r}, "
            f"url={self.url!r})"
        )


In [5]:
def scrapeAmazon(level):
    books = (
        session.query(Amazon).filter(((Amazon.rating == None) | (Amazon.rating == -level))  & (Amazon.url != None)).all()
    )
    print(len(books))
    for book in books:
        amazon = scrape(book.url, "definitions/amazon.yml")
        print(amazon, book.url)
        if amazon != None:
            rating = amazon["rating"]
            if rating != None:
                book.rating = rating.split()[0]
            reviewCount = amazon["reviewCount"]
            if reviewCount != None:
                book.review_count = reviewCount.split()[0].replace(",", "")

            if rating == None and reviewCount == None:
                try:
                    book.rating = book.rating - 1
                except: # does the same thing, but seperated for semantic reasons
                    book.rating = level - 1
                    
            session.commit()
        
        

In [8]:
#start i at 10
for i in range(20):
    try:
        scrapeAmazon(i+22)
    except:
        print("FAILED")

152
{'rating': None, 'reviewCount': None} http://www.amazon.com/Unfinished-Business-Silhouette-Intimate-Moments-ebook/dp/B005IGVS6Q?tag=NYTBSREV-20
{'rating': None, 'reviewCount': None} https://www.amazon.com/Fatal-Reckoning-Marie-Force-ebook/dp/B07CRZ3GT3?tag=NYTBSREV-20
{'rating': None, 'reviewCount': None} https://www.amazon.com/Fatal-Threat-Novel-Romantic-Suspense-ebook/dp/B01N2G5Z4Z?tag=NYTBSREV-20
{'rating': None, 'reviewCount': None} https://www.amazon.com/Dark-Humanity-Gwynn-White-ebook/dp/B01MCYGPQM?tag=NYTBSREV-20
{'rating': None, 'reviewCount': None} http://www.amazon.com/Two-Nicholas-Sparks-ebook/dp/B01ER6Z7RI?tag=NYTBSREV-20
{'rating': None, 'reviewCount': None} https://www.amazon.com/Whole-Towns-Talking-Novel/dp/140006595X?tag=NYTBSREV-20
{'rating': None, 'reviewCount': None} http://www.amazon.com/Unfinished-Business-Silhouette-Intimate-Moments-ebook/dp/B005IGVS6Q?tag=NYTBSREV-20
{'rating': None, 'reviewCount': None} https://www.amazon.com/Odessa-Sea-Dirk-Pitt-Adventure/d

In [36]:
books = session.query(Amazon).filter(((Amazon.rating == None) | (Amazon.rating <= -2))  & (Amazon.url != None)).all()

In [37]:
len(books)

1613